In [ ]:
import os
import sys

import pandas as pd
import math

from plio.io.io_bae import read_gpf, read_ipf, read_atf, save_gpf, save_ipf
from plio.utils.utils import find_in_dict
from plio.spatial.transformations import apply_isis_transformations, apply_socet_transformations, serial_numbers
import plio.io.io_controlnetwork as cn
import plio.io.isis_serial_number as sn

In [ ]:
def socet2isis(at_file, cub_file_path, extension, target_name, outpath=None):
    # Setup the at_file, path to cubes, and control network out path
    at_file = at_file
    cnet_out = os.path.split(os.path.splitext(at_file)[0])[1]
    cub_path = cub_file_path

    if(outpath):
        outpath = outpath
    else:
        outpath = os.path.split(at_file)[0]
        
    # Read in and setup the atf dict of information
    atf_dict = read_atf(at_file)
    
    # Get the gpf and ipf files using atf dict
    gpf_file = os.path.join(atf_dict['PATH'], atf_dict['GP_FILE']);
    ipf_list = [os.path.join(atf_dict['PATH'], i) for i in atf_dict['IMAGE_IPF']]
    
    # Read in the gpf file and ipf file(s) into seperate dataframes
    gpf_df = read_gpf(gpf_file)
    ipf_df = read_ipf(ipf_list)

    # Check for differences between point ids using each dataframes
    # point ids as a reference
    gpf_pt_idx = pd.Index(pd.unique(gpf_df['point_id']))
    ipf_pt_idx = pd.Index(pd.unique(ipf_df['pt_id']))

    point_diff = ipf_pt_idx.difference(gpf_pt_idx)

    if len(point_diff) != 0:
        warnings.warn("The following points found in ipf files missing from gpf file: \n\n{}. \
                      \n\nContinuing, but these points will be missing from the control network".format(list(point_diff)))
        
    # Merge the two dataframes on their point id columns
    socet_df = ipf_df.merge(gpf_df, left_on='pt_id', right_on='point_id')
    
    # Apply the transformations
    apply_socet_transformations(atf_dict, socet_df)
    
    # Define column remap for socet dataframe
    column_map = {'pt_id': 'id', 'l.': 'y', 's.': 'x',
                  'res_l': 'lineResidual', 'res_s': 'sampleResidual', 'known': 'Type',
                  'lat_Y_North': 'aprioriY', 'long_X_East': 'aprioriX', 'ht': 'aprioriZ',
                  'sig0': 'aprioriLatitudeSigma', 'sig1': 'aprioriLongitudeSigma', 'sig2': 'aprioriRadiusSigma',
                  'sig_l': 'linesigma', 'sig_s': 'samplesigma'}
    
    # Rename the columns using the column remap above
    socet_df.rename(columns = column_map, inplace=True)
    
    # Build an image and serial dict assuming the cubes will be named as the IPFs are
    image_dict = {i: i + extension for i in pd.unique(socet_df['ipf_file'])}
    serial_dict = serial_numbers(image_dict, cub_path)

    # creates the control network
    cn.to_isis(os.path.join(outpath, cnet_out + '.net'), socet_df, serial_dict, targetname = targetname)

In [ ]:
def isis2socet(cnet_path, eRadius, eccentricity, cub_path, extension, cub_list, out_gpf, adjusted_flag = False):
    pRadius = eRadius * math.sqrt(1 - (eccentricity ** 2))
    
    df = cn.from_isis(cnet_path)
    # Create cub dict to map ipf to cub
    cub_dict = {i: i + extension for i in cub_list}

    # Create serial dict to match serial to ipf
    serial_dict = {sn.generate_serial_number(os.path.join(cub_path, i + extension)): i for i in cub_list}

    # Remove duplicate columns
    # There are better ways to do this but pandas was not having it
    columns = []
    column_index = []

    for i, column in enumerate(list(df.columns)):
        if column not in columns:
            column_index.append(i)
            columns.append(column)

    df = df.iloc[:, column_index]

    # Begin translation
    # Remap the ISIS columns to socet column names
    column_map = {'id': 'pt_id', 'line': 'l.', 'sample': 's.', 
                  'lineResidual': 'res_l', 'sampleResidual': 'res_s', 'type': 'known', 
                  'aprioriLatitudeSigma': 'sig0', 'aprioriLongitudeSigma': 'sig1', 'aprioriRadiusSigma': 'sig2', 
                  'linesigma': 'sig_l', 'samplesigma': 'sig_s', 'ignore': 'stat'}

    # Depending on the adjusted flag, set the renames for columns appropriately
    if adjusted_flag:
        column_map['adjustedY'] = 'lat_Y_North'
        column_map['adjustedX'] = 'long_X_East'
        column_map['adjustedZ'] = 'ht'
    else:
        column_map['aprioriY'] = 'lat_Y_North'
        column_map['aprioriX'] = 'long_X_East'
        column_map['aprioriZ'] = 'ht'

    df.rename(columns = column_map, inplace=True)
    
    apply_isis_transformations(df, eRadius, pRadius, serial_dict, extension, cub_path)

    # Save the ipf
    save_ipf(df, os.path.split(out_gpf)[0])

    # Get the first record from each group as there all the same, put them
    # into a list, and sort it
    points = [int(i[1].index[0]) for i in df.groupby('pt_id')]
    points.sort()

    # Set the gpf_df to only the values we need and do a small rename
    gpf_df = df.iloc[points].copy()
    gpf_df.rename(columns = {'pt_id': 'point_id'}, inplace=True)

    # Save the gpf
    save_gpf(gpf_df, out_gpf)

In [ ]:
# Setup stuffs for the cub information namely the path and extension
cub_path = '/Path/to/cubs'

# Name of the target body
targetname = 'Mars'
extension = 'cub.-->extension<--'

# Path to atf file
atf_file = 'Path/to/socket/set/at_file.atf'

socet2isis(atf_file, cub_path, extension, targetname)

In [ ]:
# Setup stuffs for the cub information namely the path and extension
# along with eRadius and eccentricity
cnet = "Path/to/control/network.net"

eRadius = 3.39619000000000e+006
eccentricity = 1.08339143554195e-001

cub_path = 'Path/to/cubs'
extension = 'cub.-->extension<--'

# List of cubes to use
cub_list = ['D06_029601_1846_XN_04N224W', 
            'F05_037684_1857_XN_05N224W']

out_gpf = "/Users/adampaquette/Desktop/InSightE09_XW.gpf"

adjusted_flag = False

isis2socet(cnet, eRadius, eccentricity, cub_path, extension, cub_list, out_gpf, adjusted_flag)